In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
df = pd.read_csv("data\\train.csv")


In [ ]:

print(f"Kształt danych: {df.shape}")
display(df.head())

🔢 Kształt danych: (0, 3)


,dummy,data,1


In [ ]:
print("Informacje o zbiorze danych:")
df.info()

ℹ️ Informacje o zbiorze danych:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   dummy   0 non-null      object
 1   data    0 non-null      object
 2   1       0 non-null      object
dtypes: object(3)
memory usage: 132.0+ bytes


In [ ]:
print("Opis statystyczny danych numerycznych:")
display(df.describe())

📊 Opis statystyczny danych numerycznych:


,dummy,data,1
count,0,0,0
unique,0,0,0
top,NaN,NaN,NaN
freq,NaN,NaN,NaN


In [ ]:
missing = df.isna().sum()
missing = missing[missing > 0].sort_values(ascending=False)
print("Kolumny z brakami danych:")
display(missing)

🧱 Kolumny z brakami danych:


Series([], dtype: int64)

In [ ]:
cat_cols = df.select_dtypes(include=["object", "category"]).columns.tolist()
num_cols = df.select_dtypes(include=["number", "bool"]).columns.tolist()

print(f" Kolumny kategoryczne: {cat_cols}")
print(f" Kolumny numeryczne: {num_cols}")

In [ ]:
# Unikalne wartości w zmiennych kategorycznych
for col in cat_cols:
    print(f"{col}: {df[col].nunique()} unikalnych wartości")
    print(df[col].value_counts(dropna=False).head())
    print("")

In [ ]:
#Sprawdzenie duplikatów
duplicates = df.duplicated().sum()
print(f" Liczba duplikatów: {duplicates}")

In [ ]:
target_col = "target"  
if target_col in df.columns:
    print(f"Rozkład zmiennej docelowej: {target_col}")
    print(df[target_col].value_counts(normalize=True))
    sns.countplot(x=df[target_col])
    plt.title("Rozkład targetu")
    plt.show()

In [ ]:
# Boxploty dla wykrywania outliers
for col in num_cols:
    if df[col].nunique() > 10:  # tylko dla sensownych zmiennych ciągłych
        plt.figure(figsize=(6, 1.5))
        sns.boxplot(data=df, x=col)
        plt.title(f"Outliers in {col}")
        plt.show()

In [ ]:
#korelacje numeryczne
corr = df[num_cols].corr()
plt.figure(figsize=(10, 6))
sns.heatmap(corr, annot=True, fmt=".2f", cmap="coolwarm", square=True)
plt.title(" Korelacje między cechami numerycznymi")
plt.show()

## Pipeline - encoding

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# 1. Rozdzielenie kolumn
cat_cols = df.select_dtypes(include=["object", "category"]).columns.tolist()
num_cols = df.select_dtypes(include=["number", "bool"]).drop(columns=[target_col]).columns.tolist()

print(f" Kolumny kategoryczne: {cat_cols}")
print(f" Kolumny numeryczne: {num_cols}")

# 2. Imputacja + kodowanie kategorii
cat_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])

# 3. Imputacja + skalowanie numerów
num_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

# 4. ColumnTransformer – łączy oba pipeline’y
preprocessor = ColumnTransformer([
    ("num", num_pipeline, num_cols),
    ("cat", cat_pipeline, cat_cols)
])

## Encoding with column names

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# ✳️ Lista kolumn kategorycznych
cat_cols = df.select_dtypes(include=["object", "category"]).columns.tolist()

# 🔠 One-hot encoding
encoder = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(sparse_output=False, handle_unknown="ignore"), cat_cols)
    ],
    remainder="passthrough"  # zachowaj kolumny numeryczne bez zmian
)

X = df.drop(columns=[target_col])
y = df[target_col]

X_encoded = encoder.fit_transform(X)

print("🎯 Kształt po kodowaniu:", X_encoded.shape)


In [ ]:
# Zakładamy, że encoder został dopasowany
encoder = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(sparse_output=False, handle_unknown="ignore"), cat_cols)
    ],
    remainder="passthrough"  # zachowaj kolumny numeryczne bez zmian
)
ohe = encoder.named_transformers_["cat"]
encoded_feature_names = ohe.get_feature_names_out(cat_cols)

# Inne kolumny (numeryczne) - nie zostały zakodowane
non_cat_cols = X.drop(columns=cat_cols).columns

# Połączenie nazw wszystkich cech
all_feature_names = list(encoded_feature_names) + list(non_cat_cols)

# Konwersja do DataFrame
import pandas as pd
X_encoded_df = pd.DataFrame(X_encoded, columns=all_feature_names)


## Walidacja

In [ ]:
X = df.drop(columns=[target_col])
y = df[target_col]

X_transformed = preprocessor.fit_transform(X)

# Jeśli używasz OneHotEncoder, dane będą jako NumPy array — sprawdź shape
print("Po przetworzeniu:")
print("X shape:", X_transformed.shape)
print("y shape:", y.shape)